#  Gathering DATA

In [3]:
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import datasets

C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
all_ds = datasets.list_datasets()

C:\Users\Dell\AppData\Local\Temp\ipykernel_11272\923723365.py:1: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  all_ds = datasets.list_datasets()


In [6]:
print(len(all_ds))

47956


In [7]:
all_ds[1000:1002]

['Ishwar/Senti', 'Iskaj/dutch_corpora_parliament_processed']

In [8]:
'oscar' in all_ds

True

In [9]:
dataset = datasets.load_dataset('oscar', 'unshuffled_deduplicated_ta')

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 833101
    })
})

In [11]:
dataset['train']

Dataset({
    features: ['id', 'text'],
    num_rows: 833101
})

In [12]:
dataset['train'].features

{'id': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None)}

In [13]:
dataset['train'][0]

{'id': 0,
 'text': "பொழுது சாய்ந்து வெகு நேரமாகிவிட்டது. கூலி வேலைக்குப் போயிருந்த 'சித்தாள் ' பெண்கள் எல்லோரும் வீடு திரும்பி விட்டார்கள். இன்னும் அழகம்மாளை மட்டும் காணவில்லை.\nகுடிசைக்குள் ---தனக்கும் அழகம்மாளுக்கும் சோறு பொங்கி, குழம்பு காய்ச்சும் வேலையில் ---அடுப்புப் புகையில் குனிந்திருந்த கிழவி ஆரோக்கியம் முந்தானையில் முகத்தைத் துடைத்துக்கொண்டு, குடிசைக்கு வெளியே வந்து தலை நிமிர்ந்து பார்க்கும்போது நிலவு கிளம்பி இருந்தது.\nகுடிசையின் கதவை இழுத்து மூடிவிட்டு, தெருவில் இறங்கி நடந்தாள் ஆரோக்கியம். எதிரில் வரும் பெண்களை எல்லாம் நிறுத்தி விசாரித்தாள்.\nசேரித் தெரு முனையில் உள்ள சாயபுக் கடையில் ஒரே கும்பல்... ' அந்தக் கும்பலில் இருப்பாளோ ' '--கிழவி சாயபுக் கடையை நோக்கி ஓடினாள். கடையில் பெண்கள் கூட்டம் நிறைந்திருந்தது; அழகம்மாளைத்தான் காணோம்.\n'அட போம்மா, ஒனக்கு வேறே வேலையில்லே...நீ ஒரு பைத்தியம், அந்தப் பைத்தியத்தைத் தேடிக்கிட்டுத் திரியறே ? எங்களுக்கு வேறே வேலையில்லியா ? ' என்று எரிந்து விழுந்தான் கடைக்கார சாயபு--அவனுக்கு வியாபார மும்முரம்.\nஆமாம்; இரண்டு மாதத்துக்குமுன் அழகம்மாள் பைத

In [15]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

for sample in tqdm(dataset['train']):
  sample = sample['text'].replace('\n', ' ')
  text_data.append(sample)
  if len(text_data) == 10_000:
    with open (f'ta_{file_count}.txt', 'w', encoding = 'utf-8') as fp:
      fp.write('\n'.join(text_data))
    text_data = []
    file_count += 1

with open (f'ta_{file_count}.txt', 'w', encoding = 'utf-8') as fp:
      fp.write('\n'.join(text_data))

100%|██████████| 833101/833101 [00:43<00:00, 19372.52it/s]


# Tokenizer

In [16]:
from pathlib import Path

In [17]:
paths = [str(x) for x in Path('./').glob('*.txt')]

In [18]:
paths[0:5]

['ta_0.txt', 'ta_1.txt', 'ta_10.txt', 'ta_11.txt', 'ta_12.txt']

In [19]:
%pip install tokenizers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [20]:
from tokenizers import ByteLevelBPETokenizer

In [89]:
tokenizer = ByteLevelBPETokenizer()

In [92]:
tokenizer.train(files=paths[:8], vocab_size = 30_522, min_frequency = 2, 
                special_tokens= [
                    '<s>', '<pad>', '</s>', '<unk>', '<mask>'
                ])

In [23]:
import os
os.mkdir('TamBert')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'TamBert'

In [24]:
tokenizer.save_model('TamBert')

['TamBert\\vocab.json', 'TamBert\\merges.txt']

In [86]:
# from transformers import RobertaTokenizerFast, RobertaForMaskedLM, pipeline

# tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
# tokenizer.save_pretrained("test-tokenizer") # Only the tokenizer files are saved here

# model = RobertaForMaskedLM.from_pretrained("roberta-base")
# model.save_pretrained("test-model") # Only the model files are saved there

# fill_mask = pipeline(
#     "fill-mask",
#     model="test-model",
#     tokenizer="test-tokenizer",
# )

In [25]:
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [26]:
from transformers import RobertaTokenizerFast

In [94]:
tokenizer = RobertaTokenizerFast.from_pretrained('TamBert', return_special_tokens_mask=True, max_len = 512)

In [96]:
#tokenizer('ஆமை பாப்பா இடம்', padding= 'max_length', max_length = 512)
tokenizer('ஆமை')

{'input_ids': [0, 755, 276, 2], 'attention_mask': [1, 1, 1, 1]}

In [30]:
import torch

def mlm(tensor):
    rand = torch.rand(tensor.shape) #[0,1]
    mask_arr = (rand < 0.15) * (tensor > 2) #masking 15% of tokens with value greater than 2. As 0,1,2 are special tokens
    for i in range(tensor.shape[0]): #flattening different size tensors into the same size
        selection = torch.flatten(mask_arr[i].nonzero()) #[[2, 5, 18]] -> [2, 5, 18]
        tensor[i, selection] = 4

    return tensor

In [32]:
from tqdm.auto import tqdm

input_ids = []
mask = []
labels = []

for path in tqdm(paths[:50]):
    with open(path, 'r', encoding= 'utf-8') as f:
        lines = f.read().split('\n')

    sample = tokenizer(lines, max_length=512, padding = 'max_length', truncation=True, return_tensors = 'pt')
    labels.append(sample.input_ids)
    mask.append(sample.attention_mask)
    input_ids.append(mlm(sample.input_ids.detach().clone()))

100%|██████████| 50/50 [35:06<00:00, 42.13s/it]


In [33]:
input_ids = torch.cat(input_ids[:2])
mask = torch.cat(mask)
labels = torch.cat(labels)

In [34]:
input_ids[0][:10]

tensor([  0, 275, 299, 301, 265, 268, 265, 292,   4, 277])

In [35]:
labels[0][:10]

tensor([  0, 275, 299, 301, 265, 268, 265, 292, 270, 277])

# Building the DataLoader

In [36]:
encodings = {
    'input_ids' : input_ids,
    'attention_mask' : mask,
    'labels' : labels
}

In [37]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [38]:
dataset = Dataset(encodings)

In [39]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

# Initializing the Model

In [40]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

In [41]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config)

In [42]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [43]:
from transformers import AdamW


model.train()

optim = AdamW(model.parameters(), lr=1e-4)

C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [44]:
epochs = 1

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Epoch 0: 100%|██████████| 1250/1250 [4:40:52<00:00, 13.48s/it, loss=0.469] 


# Testing

In [45]:
from transformers import pipeline

In [76]:
fill = pipeline('fill-mask', model = 'TamBert', tokenizer = TokenizerClass.from_pretrained('C:\Users\Dell\Downloads\TamBert'))

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (4178692538.py, line 1)

In [81]:
fill = pipeline('fill-mask', model = model, tokenizer = tokenizer)

In [82]:
fill(f'தமிழ் பாடல் {fill.tokenizer.mask_token}')

[{'score': 0.5921050906181335,
  'token': 4,
  'token_str': '<mask>',
  'sequence': 'தமிழ் பாடல்'},
 {'score': 0.4040980339050293,
  'token': 2,
  'token_str': '</s>',
  'sequence': 'தமிழ் பாடல்'},
 {'score': 0.0004777309368364513,
  'token': 322,
  'token_str': ' இந',
  'sequence': 'தமிழ் பாடல் இந'},
 {'score': 0.0003516163269523531,
  'token': 0,
  'token_str': '<s>',
  'sequence': 'தமிழ் பாடல்'},
 {'score': 0.00026959486422128975,
  'token': 6,
  'token_str': '"',
  'sequence': 'தமிழ் பாடல்"'}]

In [84]:
fill(f'பல நூற்றாண்டுகளாக செய்யுள் வடிவமே தமிழ் இலக்கியங்களிலும், தத்துவங்களிலும் பயன்படுத்தப்பட்டது. {fill.tokenizer.mask_token} வடிவம் இலக்கணங்களுக்கும், செய்யுள் விளக்கம் கூறவதற்கும், சாசனங்கள் (records) பதிவு செய்வதற்கும் பயன்படுத்தப்பட்டது.')

[{'score': 0.7859432697296143,
  'token': 2,
  'token_str': '</s>',
  'sequence': 'பல நூற்றாண்டுகளாக செய்யுள் வடிவமே தமிழ் இலக்கியங்களிலும், தத்துவங்களிலும் பயன்படுத்தப்பட்டது. வடிவம் இலக்கணங்களுக்கும், செய்யுள் விளக்கம் கூறவதற்கும், சாசனங்கள் (records) பதிவு செய்வதற்கும் பயன்படுத்தப்பட்டது.'},
 {'score': 0.1056893840432167,
  'token': 4,
  'token_str': '<mask>',
  'sequence': 'பல நூற்றாண்டுகளாக செய்யுள் வடிவமே தமிழ் இலக்கியங்களிலும், தத்துவங்களிலும் பயன்படுத்தப்பட்டது. வடிவம் இலக்கணங்களுக்கும், செய்யுள் விளக்கம் கூறவதற்கும், சாசனங்கள் (records) பதிவு செய்வதற்கும் பயன்படுத்தப்பட்டது.'},
 {'score': 0.005548512097448111,
  'token': 6,
  'token_str': '"',
  'sequence': 'பல நூற்றாண்டுகளாக செய்யுள் வடிவமே தமிழ் இலக்கியங்களிலும், தத்துவங்களிலும் பயன்படுத்தப்பட்டது." வடிவம் இலக்கணங்களுக்கும், செய்யுள் விளக்கம் கூறவதற்கும், சாசனங்கள் (records) பதிவு செய்வதற்கும் பயன்படுத்தப்பட்டது.'},
 {'score': 0.004838256631046534,
  'token': 7,
  'token_str': '#',
  'sequence': 'பல நூற்றாண்டுகளாக செய்யுள் வ

In [83]:
fill(f'தமிழ் {fill.tokenizer.mask_token}')

[{'score': 0.7290130853652954,
  'token': 4,
  'token_str': '<mask>',
  'sequence': 'தமிழ்'},
 {'score': 0.269315630197525,
  'token': 2,
  'token_str': '</s>',
  'sequence': 'தமிழ்'},
 {'score': 0.0006286358111537993,
  'token': 322,
  'token_str': ' இந',
  'sequence': 'தமிழ் இந'},
 {'score': 0.0001477646583225578,
  'token': 0,
  'token_str': '<s>',
  'sequence': 'தமிழ்'},
 {'score': 0.00013986047997605056,
  'token': 742,
  'token_str': ' ரச',
  'sequence': 'தமிழ் ரச'}]

In [91]:
fill(f'பல நூற்றாண்டுகளாக செய்யுள் {fill.tokenizer.mask_token} தமிழ் இலக்கியங்களிலும், தத்துவங்களிலும் பயன்படுத்தப்பட்டது ')

[{'score': 0.2507525086402893,
  'token': 4,
  'token_str': '<mask>',
  'sequence': 'பல நூற்றாண்டுகளாக செய்யுள் தமிழ் இலக்கியங்களிலும், தத்துவங்களிலும் பயன்படுத்தப்பட்டது '},
 {'score': 0.04196109622716904,
  'token': 10,
  'token_str': '&',
  'sequence': 'பல நூற்றாண்டுகளாக செய்யுள்& தமிழ் இலக்கியங்களிலும், தத்துவங்களிலும் பயன்படுத்தப்பட்டது '},
 {'score': 0.03249293938279152,
  'token': 92,
  'token_str': 'x',
  'sequence': 'பல நூற்றாண்டுகளாக செய்யுள்x தமிழ் இலக்கியங்களிலும், தத்துவங்களிலும் பயன்படுத்தப்பட்டது '},
 {'score': 0.020259948447346687,
  'token': 284,
  'token_str': 'கள',
  'sequence': 'பல நூற்றாண்டுகளாக செய்யுள்கள தமிழ் இலக்கியங்களிலும், தத்துவங்களிலும் பயன்படுத்தப்பட்டது '},
 {'score': 0.014728732407093048,
  'token': 7,
  'token_str': '#',
  'sequence': 'பல நூற்றாண்டுகளாக செய்யுள்# தமிழ் இலக்கியங்களிலும், தத்துவங்களிலும் பயன்படுத்தப்பட்டது '}]